In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import seaborn as sb
sb.set_style('darkgrid')

from glob import glob
import matplotlib.pyplot as plt
import os
import random
import cv2

import gzip
import nibabel as nb

In [2]:
MriFiles = glob('/Volumes/DISK_IMG/10*/*T1w.nii.gz')
SosFiles = glob('/Volumes/DISK_IMG/10*/*Vp.nii.gz')

MriIm = [] # array of full images containing header and data of MRIs
MriHdr = [] # array of only the headers of MRIs
MriData = [] # array of only the data of MRIs

# similarly, for speed of sound maps
SosIm = []
SosHdr = []
SosData = []
for i in range(len(MriFiles)):
    MriIm.append(nb.load(MriFiles[i]))
    MriHdr.append(MriIm[i].header)
    MriData.append(MriIm[i].get_fdata())

    SosIm.append(nb.load(SosFiles[i]))
    SosHdr.append(SosIm[i].header)
    SosData.append(SosIm[i].get_fdata())

In [6]:
# put all the scans of all MRIs as members of a dataframe so that each cell of each row is a whole image
MriDf = pd.DataFrame()
for scan in range(len(MriData)):
    Mri = MriData[scan]
    for slice in range(len(Mri[1,1,:])):
        temp = {'img' : [Mri[:,:,slice]]}
        tempDf = pd.DataFrame.from_records(temp)
        MriDf = MriDf.append(tempDf)
print("Length of MRI dataset: " + str(MriDf.shape[0]))
# same for SoS
SosDf = pd.DataFrame()
for scan in range(len(SosData)):
    Sos = SosData[scan]
    for slice in range(len(Sos[1,1,:])):
        temp = {'img' : [Sos[:,:,slice]]}
        tempDf = pd.DataFrame.from_records(temp)
        SosDf = SosDf.append(tempDf)
print("Length of SoS dataset: " + str(SosDf.shape[0]))

Length of MRI dataset: 1600
Length of SoS dataset: 1600


In [55]:
# must vectorize each cell of the df (each (n,m) matrix) so structure is:
# X has form (n.m, 2) (col of 1s and col of values)
# y has form (n.m, 1)
vec_MriDf = pd.DataFrame()
for row in range(MriDf.shape[0]):
    temp = pd.DataFrame({'' : np.array([MriDf.iloc[row][0].flatten()]).tolist()})
    vec_MriDf = pd.concat([vec_MriDf,temp])

In [79]:
"""
making it so each column is a pixel. eg.
       MRI              becomes              0    1    2
 0  [a, b, c]                           0    a    b    c
 1  [d, e, f]                           1    d    e    f
 2  [g, h, i]                           2    g    h    i
 3  [j, k, l]                           3    j    k    l
       ...                                       ...

"""
MriDf2 = pd.DataFrame(np.nan, index=[x for x in range(vec_MriDf.shape[0])], columns=[x for x in range(len(vec_MriDf.iloc[0][0]))]) #1600x256
for slice in range(vec_MriDf.shape[0]):
      for pixel in range(len(vec_MriDf.iloc[0][0])):
            MriDf2.iloc[slice][pixel] = vec_MriDf.iloc[0][0][pixel]

KeyboardInterrupt: 

In [78]:
vec_MriDf.shape

(1600, 1)